## Data preprocessing
Ricette definite in un formato semistrutturato, raggruppamento degli step e ingredienti in un unica stringa, funzioni di utilità

### Funzioni util e import

In [66]:
from pathlib import Path
import pandas as pd
from IPython.display import display
from ast import literal_eval
import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer


def string_recipe(i):
    return dataset.iloc[i]['title'] + "\n\n" + dataset.iloc[i]['ingredients'] + "\n\n" + dataset.iloc[i]['step'] 

### Caricamento del dataset

In [67]:
dataset = pd.read_csv(
    Path("../data/test_dataset.csv").resolve(), 
    index_col=[0], 
    names=["index", "title","ingredients","step"], 
    usecols=[0,1,2,3]
    )

for index in range(len(dataset)):
    dataset.iloc[index]['ingredients'] = ".\n".join(literal_eval(dataset.iloc[index]['ingredients']))
    dataset.iloc[index]['step'] = " ".join(literal_eval(dataset.iloc[index]['step']))

display(dataset.head())
print(string_recipe(10))


,title,ingredients,step
index,,,
1,Jewell Ball'S Chicken,"1 small jar chipped beef, cut up.\n4 boned chi...",Place chipped beef on bottom of baking dish. P...
2,Creamy Corn,2 (16 oz.) pkg. frozen corn.\n1 (8 oz.) pkg. c...,"In a slow cooker, combine all ingredients. Cov..."
3,Chicken Funny,1 large whole chicken.\n2 (10 1/2 oz.) cans ch...,Boil and debone chicken. Put bite size pieces ...
4,Reeses Cups(Candy),1 c. peanut butter.\n3/4 c. graham cracker cru...,Combine first four ingredients and press in 13...
5,Cheeseburger Potato Soup,6 baking potatoes.\n1 lb. of extra lean ground...,Wash potatoes; prick several times with a fork...


Buckeye Candy

1 box powdered sugar.
8 oz. soft butter.
1 (8 oz.) peanut butter.
paraffin.
12 oz. chocolate chips

Mix sugar, butter and peanut butter. Roll into balls and place on cookie sheet. Set in freezer for at least 30 minutes. Melt chocolate chips and paraffin in double boiler. Using a toothpick, dip balls 3/4 of way into chocolate chip and paraffin mixture to make them look like buckeyes.


### Estrazione delle abbreviazioni

Fase iniziale di ritrovamento del set di abbreviazioni

In [17]:
abbrv_dataset = pd.read_csv(
    Path("../data/test_dataset.csv").resolve(), 
    index_col=[0], 
    names=["index", "title","ingredients","step"], 
    usecols=[0,1,2,3]
    )

abbrv = set()
for index in range(len(dataset)):
    abbrv_dataset.iloc[index]['ingredients'] = " ".join(literal_eval(abbrv_dataset.iloc[index]['ingredients']))
    for element in re.findall(r"[A-Za-z]*\.", abbrv_dataset.iloc[index]['ingredients']):
        abbrv.add(element)
    
print(abbrv)

{'tsp.', 'Tbsp.', 'lb.', 'pt.', 'gal.', 'sq.', 'c.', 'pkg.', 'No.', 'tbsp.', 'qt.', 'oz.'}


Rimozione delle abbreviazioni in quanto possono essere dannose per il processo di tokenizzazione. Es pkg. ---> package

In [19]:
def expand_abbreviations(ingredients_string):
    __ABBREVIATIONS__ = {
        'pkg.'  :   'package',
        'tsb.'  :   'tablespoon',
        'no.'   :   'number',
        'pt.'   :   'pint',
        'no.'   :   'number',
        'gal.'  :   'gallon',
        'tbsp.' :   'tablespoon',
        'sq.'   :   'square',
        'oz.'   :   'ounce',
        'lb.'   :   'pound',
        'qt.'   :   'quart',
        'c.'    :   'cup',
        'tsp.'  :   'teaspoon'
    }
    for item, value in __ABBREVIATIONS__.items():
        ingredients_string = ingredients_string.lower().replace(item, value)
    return ingredients_string


for index in range(len(dataset)):
    dataset.iloc[index]['ingredients'] = expand_abbreviations(dataset.iloc[index]['ingredients'])

### Sentence splitting
Il contenuto delle colonne 'ingredients' e 'step' verrà suddiviso in frasi. In precedenza i periodi contenuti nelle singole celle sono stati formattati in modo tale da renderli riconoscibili e facilmente suddivisibili in frasi ben separate.

In [68]:
for index in range(len(dataset)):
     dataset.iloc[index]['ingredients'] = (sent_tokenize(dataset.iloc[index]['ingredients']))
print(dataset.iloc[10]['ingredients'])

['1 box powdered sugar.', '8 oz.', 'soft butter.', '1 (8 oz.)', 'peanut butter.', 'paraffin.', '12 oz.', 'chocolate chips']


In [69]:
for index in range(len(dataset)):
     dataset.iloc[index]['step'] = (sent_tokenize(dataset.iloc[index]['step']))
print(dataset.iloc[10]['step'])

['Mix sugar, butter and peanut butter.', 'Roll into balls and place on cookie sheet.', 'Set in freezer for at least 30 minutes.', 'Melt chocolate chips and paraffin in double boiler.', 'Using a toothpick, dip balls 3/4 of way into chocolate chip and paraffin mixture to make them look like buckeyes.']


### Stop words removal
Nella colonna 'step' troviamo una serie di passaggi da compiere per creare la ricetta. Questi passaggi sono scritti in linguaggio naturale e possono essere semplificati rimuovendo delle parole dette stop words.

In [70]:
stop_words = set(stopwords.words('english'))
tokenizer = RegexpTokenizer(r'\w+')

for index in range(len(dataset)):
    step = []
    for sent in dataset.iloc[index]['step']:
        words = tokenizer.tokenize(sent)
        clean_sent = [w for w in words if w not in stop_words]
        clean_sent = ' '.join(clean_sent)
        step.append(clean_sent)
    dataset.iloc[index]['step'] = step
print(dataset.iloc[10]['step'])


['Mix sugar butter peanut butter', 'Roll balls place cookie sheet', 'Set freezer least 30 minutes', 'Melt chocolate chips paraffin double boiler', 'Using toothpick dip balls 3 4 way chocolate chip paraffin mixture make look like buckeyes']


### Stemming e Lemming
Questi due processi potrebbero portare valore all'analisi del dominio. Il codice per entrambi viene proposto qui.

In [71]:
lemmatizer = WordNetLemmatizer()

for index in range(len(dataset)):
    step = []
    for sent in dataset.iloc[index]['step']:
        words = tokenizer.tokenize(sent)
        stem_sent = [lemmatizer.lemmatize(w) for w in words]
        stem_sent = ' '.join(stem_sent)
        step.append(stem_sent)
    dataset.iloc[index]['step'] = step
print(dataset.iloc[10]['step'])

['Mix sugar butter peanut butter', 'Roll ball place cookie sheet', 'Set freezer least 30 minute', 'Melt chocolate chip paraffin double boiler', 'Using toothpick dip ball 3 4 way chocolate chip paraffin mixture make look like buckeye']


In [ ]:
stemmer = PorterStemmer()

for index in range(len(dataset)):
    step = []
    for sent in dataset.iloc[index]['step']:
        words = tokenizer.tokenize(sent)
        stem_sent = [stemmer.stem(w) for w in words]
        stem_sent = ' '.join(stem_sent)
        step.append(stem_sent)
    dataset.iloc[index]['step'] = step
print(dataset.iloc[10]['step'])

### POS-Tagging
Questa operazione ci permette di riconoscere le parti del testo. !!!Attualmente vengono solo stampate, ma in seguito le utilizzeremo a modo!!!

In [79]:
#POS-Tagging per gli ingredienti

for index in range(len(dataset)):
    for sent in dataset.iloc[index]['ingredients']:
        words = tokenizer.tokenize(sent)
        tagged = nltk.pos_tag(words)
        print(tagged)

[('1', 'CD'), ('small', 'JJ'), ('jar', 'NN'), ('chipped', 'VBD'), ('beef', 'NN'), ('cut', 'NN'), ('up', 'RP')]
[('4', 'CD'), ('boned', 'VBD'), ('chicken', 'NN'), ('breasts', 'NNS')]
[('1', 'CD'), ('can', 'MD'), ('cream', 'VB'), ('of', 'IN'), ('mushroom', 'NN'), ('soup', 'NN')]
[('1', 'CD'), ('carton', 'NN'), ('sour', 'NN'), ('cream', 'NN')]
[('2', 'CD'), ('16', 'CD'), ('oz', 'NN')]
[('pkg', 'NN')]
[('frozen', 'JJ'), ('corn', 'NN')]
[('1', 'CD'), ('8', 'CD'), ('oz', 'NN')]
[('pkg', 'NN')]
[('cream', 'NN'), ('cheese', 'NN'), ('cubed', 'VBD')]
[('1', 'CD'), ('3', 'CD'), ('c', 'NNS'), ('butter', 'NN'), ('cubed', 'VBD')]
[('1', 'CD'), ('2', 'CD'), ('tsp', 'NN')]
[('garlic', 'JJ'), ('powder', 'NN')]
[('1', 'CD'), ('2', 'CD'), ('tsp', 'NN')]
[('salt', 'NN')]
[('1', 'CD'), ('4', 'CD'), ('tsp', 'NN')]
[('pepper', 'NN')]
[('1', 'CD'), ('large', 'JJ'), ('whole', 'JJ'), ('chicken', 'NN')]
[('2', 'CD'), ('10', 'CD'), ('1', 'CD'), ('2', 'CD'), ('oz', 'NN')]
[('cans', 'NNS'), ('chicken', 'VBP'), ('gr

In [78]:
#POS-Tagging per gli step

for index in range(len(dataset)):
    for sent in dataset.iloc[index]['step']:
        words = tokenizer.tokenize(sent)
        tagged = nltk.pos_tag(words)
        print(tagged)

[('Place', 'NN'), ('chipped', 'VBD'), ('beef', 'NN'), ('bottom', 'NN'), ('baking', 'VBG'), ('dish', 'NN')]
[('Place', 'NNP'), ('chicken', 'VBP'), ('top', 'JJ'), ('beef', 'NN')]
[('Mix', 'NNP'), ('soup', 'NN'), ('cream', 'NN'), ('together', 'RB'), ('pour', 'VBP'), ('chicken', 'NN')]
[('Bake', 'NNP'), ('uncovered', 'VBD'), ('275', 'CD'), ('3', 'CD'), ('hour', 'NN')]
[('In', 'IN'), ('slow', 'JJ'), ('cooker', 'NN'), ('combine', 'NN'), ('ingredient', 'NN')]
[('Cover', 'NNP'), ('cook', 'VBD'), ('low', 'JJ'), ('4', 'CD'), ('hour', 'NN'), ('heated', 'VBN'), ('cheese', 'NN'), ('melted', 'VBD')]
[('Stir', 'NNP'), ('well', 'RB'), ('serving', 'VBG')]
[('Yields', 'NNS'), ('6', 'CD'), ('serving', 'VBG')]
[('Boil', 'NNP'), ('debone', 'NN'), ('chicken', 'NN')]
[('Put', 'VB'), ('bite', 'JJ'), ('size', 'NN'), ('piece', 'NN'), ('average', 'JJ'), ('size', 'NN'), ('square', 'JJ'), ('casserole', 'JJ'), ('dish', 'NN')]
[('Pour', 'NNP'), ('gravy', 'NN'), ('cream', 'NN'), ('mushroom', 'NN'), ('soup', 'NN'), ('